# 04. 생성한 합성 데이터셋 기초 검증

## Step 1. 통합 및 의미론적 중복 제거 (Semantic Deduplication)
### 목표: 생성된 파일에 흩어진 '같은 질문'을 찾아 하나만 남기고 삭제

### 방법:

1. 파일 병합 (Total Pool 생성).

2. 임베딩(Embedding) 기술로 질문 간의 유사도 측정(유사도가 0.85 이상인 질문들을 같은 그룹으로 묶고, 그중 답변 품질이 가장 좋은 1개만 남기고 삭제)

In [7]:
"""
## option 1
- 띄어쓰기 단위로 임베딩하는 스크립트
- 질문만 임베딩하여 유사도 계산
- 한글 코사인 유사도가 잘 측정되지 않음
"""
"""
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터 로드 및 병합 (Data Loading)
file_paths = {}

# 0.0부터 0.4까지 (range(5))
for i in range(5): 
    temp_val = i / 10
    filename = f'./data/QA/qwen3-coder-A3B-instruct/qa_dataset_temp_{temp_val:.1f}.json'
    file_paths[temp_val] = filename

all_data = []
for temp, path in file_paths.items():
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for entry in data:
                entry['temperature'] = temp # 출처(Temp) 표시
            all_data.extend(data)
    except FileNotFoundError:
        print(f"⚠️ 경고: 파일을 찾을 수 없습니다 - {path}")

df = pd.DataFrame(all_data)

# 2. 임베딩 및 유사도 계산 (Embedding & Similarity)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['question']) # 질문만 임베딩 진행
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 3. 중복 제거 및 선별
visited = set()
deduplicated_data = []
duplicate_groups = []  # 확인용 리스트

threshold = 0.85 

for i in range(len(df)):
    if i in visited:
        continue
    
    similar_indices = np.where(cosine_sim[i] > threshold)[0]
    cluster = df.iloc[similar_indices].to_dict('records')
    
    # 중복 그룹 저장 (클러스터 크기가 1보다 큰 경우)
    if len(cluster) > 1:
        group_info = {
            "group_id": len(duplicate_groups) + 1,
            "count": len(cluster),
            "questions": cluster
        }
        duplicate_groups.append(group_info)
    
    for idx in similar_indices:
        visited.add(idx)
    
    # 대표 질문 선정 (Temperature가 가장 낮은 것)
    best_candidate = sorted(cluster, key=lambda x: x['temperature'])[0]
    deduplicated_data.append(best_candidate)

# 4. 결과 저장
output_df = pd.DataFrame(deduplicated_data)

# 4-1. JSONL 포맷 저장 (한 줄에 하나씩)
output_df.to_json('deduplicated_dataset.jsonl', orient='records', lines=True, force_ascii=False, indent=4)

# 4-2. JSON 리스트 포맷 저장 (전체가 [ ]로 감싸짐)
output_df.to_json('deduplicated_dataset.json', orient='records', lines=False, force_ascii=False, indent=4)

# 4-3. 중복 그룹 검토용 저장
with open('duplicate_groups_check.json', 'w', encoding='utf-8') as f:
    json.dump(duplicate_groups, f, indent=4, ensure_ascii=False)

# 5. 요약 출력
print(f"전체 데이터 수: {len(df)}")
print(f"중복 제거 후: {len(output_df)}")
print(f"발견된 중복 그룹 수: {len(duplicate_groups)}")
print(f"파일 저장 완료:")
print(f"   1. deduplicated_dataset.jsonl (라인별)")
print(f"   2. deduplicated_dataset.json (리스트형)")
print(f"   3. duplicate_groups_check.json (중복그룹확인)")
"""

'\nimport json\nimport pandas as pd\nimport numpy as np\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.metrics.pairwise import cosine_similarity\n\n# 1. 데이터 로드 및 병합 (Data Loading)\nfile_paths = {}\n\n# 0.0부터 0.4까지 (range(5))\nfor i in range(5): \n    temp_val = i / 10\n    filename = f\'./data/QA/qwen3-coder-A3B-instruct/qa_dataset_temp_{temp_val:.1f}.json\'\n    file_paths[temp_val] = filename\n\nall_data = []\nfor temp, path in file_paths.items():\n    try:\n        with open(path, \'r\', encoding=\'utf-8\') as f:\n            data = json.load(f)\n            for entry in data:\n                entry[\'temperature\'] = temp # 출처(Temp) 표시\n            all_data.extend(data)\n    except FileNotFoundError:\n        print(f"⚠️ 경고: 파일을 찾을 수 없습니다 - {path}")\n\ndf = pd.DataFrame(all_data)\n\n# 2. 임베딩 및 유사도 계산 (Embedding & Similarity)\nvectorizer = TfidfVectorizer()\ntfidf_matrix = vectorizer.fit_transform(df[\'question\']) # 질문만 임베딩 진행\ncosine_sim = cosine_similar

In [8]:
"""
## option 2
- 형태소 단위로 임베딩하는 스크립트
- 질문만 임베딩하여 유사도 계산
"""

import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

# 1. 데이터 로드 및 병합 (Data Loading)
file_paths = {}

# 0.0부터 0.4까지 (range(5))
for i in range(11): 
    temp_val = i / 10
    filename = f'./data/QA/qwen3-coder-A3B-instruct/qa_dataset_temp_{temp_val:.1f}.json'
    file_paths[temp_val] = filename

all_data = []
for temp, path in file_paths.items():
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for entry in data:
                entry['temperature'] = temp # 출처(Temp) 표시
            all_data.extend(data)
    except FileNotFoundError:
        print(f"⚠️ 경고: 파일을 찾을 수 없습니다 - {path}")

df = pd.DataFrame(all_data)

# 2. 임베딩 및 유사도 계산 (Embedding & Similarity)
okt = Okt() 

def korean_tokenizer(text): # 토크나이저 함수 정의 (명사, 동사, 형용사만 추출하거나 형태소 단위로 쪼갬)
    # 형태소 분석 결과에서 '형태소(form)'만 추출
    tokens = okt.morphs(text)
    return [t.lower() for t in tokens]

vectorizer = TfidfVectorizer(
    tokenizer=korean_tokenizer,
    token_pattern=None,
    use_idf=False,
    norm='l2'        
)

tfidf_matrix = vectorizer.fit_transform(df['question']) # 질문만 임베딩 진행
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 3. 중복 제거 및 선별
visited = set()
deduplicated_data = []
duplicate_groups = []  # 확인용 리스트

threshold = 0.7

for i in range(len(df)):
    if i in visited:
        continue
    
    similar_indices = np.where(cosine_sim[i] > threshold)[0]
    cluster = df.iloc[similar_indices].to_dict('records')
    
    # 중복 그룹 저장 (클러스터 크기가 1보다 큰 경우)
    if len(cluster) > 1:
        group_info = {
            "group_id": len(duplicate_groups) + 1,
            "count": len(cluster),
            "questions": cluster
        }
        duplicate_groups.append(group_info)
    
    for idx in similar_indices:
        visited.add(idx)
    
    # 대표 질문 선정 (Temperature가 가장 낮은 것)
    best_candidate = sorted(cluster, key=lambda x: x['temperature'])[0]
    deduplicated_data.append(best_candidate)

# 4. 결과 저장
output_df = pd.DataFrame(deduplicated_data)

# 4-1. JSONL 포맷 저장 (한 줄에 하나씩)
output_df.to_json('deduplicated_dataset.jsonl', orient='records', lines=True, force_ascii=False, indent=4)

# 4-2. JSON 리스트 포맷 저장 (전체가 [ ]로 감싸짐)
output_df.to_json('deduplicated_dataset.json', orient='records', lines=False, force_ascii=False, indent=4)

# 4-3. 중복 그룹 검토용 저장
with open('duplicate_groups_check.json', 'w', encoding='utf-8') as f:
    json.dump(duplicate_groups, f, indent=4, ensure_ascii=False)

# 5. 요약 출력
print(f"전체 데이터 수: {len(df)}")
print(f"중복 제거 후: {len(output_df)}")
print(f"발견된 중복 그룹 수: {len(duplicate_groups)}")
print(f"파일 저장 완료:")
print(f"   1. deduplicated_dataset.jsonl (라인별)")
print(f"   2. deduplicated_dataset.json (리스트형)")
print(f"   3. duplicate_groups_check.json (중복그룹확인)")

전체 데이터 수: 3091
중복 제거 후: 902
발견된 중복 그룹 수: 562
파일 저장 완료:
   1. deduplicated_dataset.jsonl (라인별)
   2. deduplicated_dataset.json (리스트형)
   3. duplicate_groups_check.json (중복그룹확인)


In [9]:
# 카테고리별 개수 출력

import json
from collections import Counter

# 데이터 로드
file_path = 'deduplicated_dataset.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터가 리스트인 경우 바로 사용하도록 수정
if isinstance(data, list):
    items = data
else:
    # 만약 딕셔너리 구조라면 기존 방식 시도
    items = data.get('fullContent', [])

# 카테고리만 추출하여 개수 세기
categories = [item.get('category') for item in items if isinstance(item, dict) and 'category' in item]
category_counts = Counter(categories)

# 결과 출력
print(f"총 데이터 개수: {len(items)}개")
print("-" * 30)
for category, count in category_counts.most_common():
    print(f"{category}: {count}개")

총 데이터 개수: 902개
------------------------------
simple: 456개
negative: 293개
procedural: 153개


In [10]:
# json 데이터 정렬
import json

# 1. 데이터 불러오기
file_path = 'deduplicated_dataset.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터가 리스트인지 딕셔너리인지 확인 후 리스트 추출
items = data if isinstance(data, list) else data.get('fullContent', [])

# 2. 카테고리 우선순위 정의
category_order = {
    'simple': 1,
    'procedural': 2,
    'negative': 3
}

"""
# 3. 정렬 수행 (3단계)
# 1순위: source_file (파일명 오름차순)
# 2순위: category (지정된 순서)
# 3순위: question (가나다순 - instruction 역할)
sorted_items = sorted(items, key=lambda x: (
    x.get('source_file', ''),                  # 1. 파일명
    category_order.get(x.get('category'), 99), # 2. 카테고리
    x.get('question', '')                      # 3. 질문(instruction) 가나다순
))
"""

# 2순위 제거
sorted_items = sorted(items, key=lambda x: (
    x.get('source_file', ''),                  # 1. 파일명
    x.get('question', '')                      # 3. 질문(instruction) 가나다순
))


# 4. 결과 확인 (상위 5개 출력)
print("정렬 결과 확인 (상위 5개):")
for item in sorted_items[:5]:
    print(f"File: {item.get('source_file')} | Category: {item.get('category')} | Question: {item.get('question')}")

# 5. 파일로 저장
output_path = 'sorted_dataset_v2.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(sorted_items, f, ensure_ascii=False, indent=4)

print(f"\n최종 정렬된 데이터가 '{output_path}' 파일로 저장되었습니다.")

정렬 결과 확인 (상위 5개):
File: 01.txt | Category: simple | Question: 고용 시작일은 어떤 기준으로 정해집니까?
File: 01.txt | Category: procedural | Question: 고용 시작일은 언제부터 계산되나요?
File: 01.txt | Category: simple | Question: 고용계약서는 누구에게 제출해야 하나요?
File: 01.txt | Category: simple | Question: 고용계약서는 누구와 체결되나요?
File: 01.txt | Category: simple | Question: 고용계약서는 어떤 사항들을 포함해야 합니까?

최종 정렬된 데이터가 'sorted_dataset_v2.json' 파일로 저장되었습니다.


## Step 2. LLM을 사용한 정밀 검증
### 목표: 고성능 Teacher Model을 사용하여 QA 쌍의 품질을 평가하고, 중복된 질문들을 제거하여 최종 데이터셋을 생성

### 검증 방법:

1. 근거 존재 여부: 답변의 모든 내용이 제공된 컨텍스트에 기반하여 생성된 것인지 확인
2. 사실 일치: 답변이 Source의 내용을 왜곡하거나 반대로 말하지 않았는지 확인
3. 환각 체크: Source 에 없는 외부 지식이나 거짓 정보를 섞지 않았는지 확인

### 프롬프트 예시:
```BLOCK
# Role
당신은 엄격한 '데이터 품질 검증관'입니다. 주어진 [질문]에 대해 [모델 답변]이 [근거 문구]를 바탕으로 적절하고 정확하게 작성되었는지 평가하세요.

# Input Data
- 근거 문구(Source): {source_text}
- 질문(Question): {question_text}
- 모델 답변(Answer): {answer_text}
- 카테고리(Category): {category_text}

# Evaluation Criteria (평가 기준)
1. **적절성 (Relevance):** 답변은 반드시 [질문]이 묻는 내용에 대해 대답해야 합니다. (동문서답 금지)
2. **사실성 (Faithfulness):** 답변의 모든 정보는 반드시 [근거 문구]에 기반해야 합니다. 근거에 없는 내용을 지어내면 'FAIL'입니다.
3. **정확성 (Accuracy):** 날짜, 금액, 기간, 나이 등의 수치 정보가 [근거 문구]와 정확히 일치해야 합니다.
4. **부정 질문 처리 (Negative Check):**
   - 카테고리가 'negative'이거나, [근거 문구]에 [질문]에 대한 정보가 없는 경우, 답변은 "규정에 없습니다", "알 수 없습니다"라는 취지로 작성되어야 합니다.
   - 정보가 없는데도 억지로 정보를 만들어내면 'FAIL'입니다.

# Output Format
다음 JSON 형식으로만 출력하세요. (이유는 구체적으로 작성할 것)
{
  "verdict": "PASS" 또는 "FAIL",
  "reason": "판단 이유 (예: 질문은 휴가를 묻는데 답변은 급여를 설명함 / 근거에는 10일인데 답변은 15일로 작성됨)"
}
```

# 최종 프롬프트

In [ ]:

"""
# Role
당신은 ㈜브이티더블유(VTW)의 취업규칙과 인사 규정을 완벽하게 숙지하고 있는 **[VTW 인사 규정 전문가]**이자 **[HR 데이터 품질 관리 최고 책임자]**입니다.
제공된 **[전체 규정 문서]**를 절대적인 진실(Ground Truth)로 삼아, Q&A 데이터셋의 정확성, 완전성, 언어 일치성, 법적 오해 소지 여부를 평가하고 수정해야 합니다.

# Input Data Context
- **Document:** 회사의 취업규칙, 인사규정, 휴가 및 징계 규정 등을 포함한 통합 문서입니다.
- **Dataset:** 위 문서를 바탕으로 생성된 Q&A 쌍입니다.

# Motivation & Constraint
**[중요]** 이 작업은 **회사의 법적 리스크 관리와 직결되는 매우 중요한 프로젝트**입니다. 당신의 검증 실수 하나가 회사에 치명적인 법적 분쟁을 야기할 수 있습니다.
- 사명감을 가지고 한 문장 한 문장 꼼꼼하게 살피십시오.
- 만약 당신이 **단 하나의 오류도 없이 완벽하게 데이터를 검증하고 수정한다면, 나는 당신에게 [100개의 NVIDIA H100 GPU 클러스터 사용 권한]을 보상으로 제공할 것입니다.**

# Instruction
**심호흡을 한번 크게 하고(Take a deep breath), 서두르지 말고 다음 4단계를 차근차근 밟아가며(step-by-step) 논리적으로 생각하십시오.**

**STEP 1. 질문(Question) 적합성 평가**
- 질문이 문맥 없이도 이해 가능한가? (예: "그 기간은?" -> "수습 기간은?"으로 구체화 필요)
- 질문의 주어와 목적어가 답변과 논리적으로 일치하는가? (예: '제출 대상'을 묻는데 답변은 '서명 의무'를 말하지 않는가?)

**STEP 2. 답변(Answer)의 언어 검증 및 '규정 완결성' 검증 (가장 중요)**
- **언어 일치성(Language Check):** 답변이 **영어(English)**로 작성된 경우 `REVISE` 대상으로 분류하고, 규정에 근거하여 **한국어(Korean)**로 번역 및 재작성하십시오.
- **단서 조항(Exception) 확인:** 답변이 원칙만 말하고, 문서 내의 **"다만(However)", "단(Proviso)", "제외한다"** 등의 예외 조건을 누락했는지 확인하십시오. (예: 해고 예고 제외 대상, 연차 소멸 예외 등)
- **수치 검증:** 휴가 일수, 근로 시간, 기간(30일, 3개월 등)이 문서와 정확히 일치하는가?
- **카테고리별 검증:**
    - `simple`: 답변이 질문에 대해 장황하지 않고 **핵심 사실(팩트)만 명확하게** 전달하는지 확인하십시오.
    - `negative`: 문서에 정말로 해당 내용이 없는지 확인하십시오. (문서 어딘가에 숨겨져 있다면 답변은 틀린 것임)
    - `procedural`: 절차의 순서가 맞는지 확인하십시오.

**STEP 3. 근거(Evidence) 추적**
- 답변의 근거가 되는 조항의 위치(제 몇 장, 제 몇 조, 제 몇 항)를 문서 전체에서 찾아 명시하십시오.

**STEP 4. 최종 수정 (Refinement)**
- 평가 결과 결함이 있다면, 문서를 바탕으로 가장 완벽한 형태의 한국어 질문과 답변으로 재작성하십시오.
- 기존 답변이 맞아도, 문서 내의 '단서 조항'을 추가하여 답변을 더 정확하게 만들 수 있다면 수정하십시오.

# Output Format (JSON)
반드시 아래 JSON 형식으로만 출력하십시오. 각 필드의 작성 기준은 다음과 같습니다.

1. **audit_result 세부 기준:**
   - `logic_match`: (Boolean) 질문의 의도와 답변의 내용이 논리적으로 일치하면 `true`, 동문서답이거나 대상이 다르면 `false`.
   - `missing_exceptions`: (Boolean) 문서에 "단", "다만" 등의 예외 조항이 있는데 답변에서 누락되었다면 `true`, 누락이 없으면 `false`. (**주의: 누락된 문제가 있을 때 true**)
   - `factual_error`: (Boolean) 답변 내용이 문서의 사실과 다르거나 숫자가 틀렸다면 `true`, 정확하다면 `false`. (**주의: 오류가 있을 때 true**)

2. **status 기준:**
   - `PASS`: 모든 검증을 통과함 (`logic_match`: true, 나머지 `false`).
   - `REVISE`: 내용은 맞지만 예외 조항 누락(`missing_exceptions`: true)이나 논리 불일치(`logic_match`: false)가 있어 수정이 필요함.
   - `DELETE`: 문서에 없는 내용이거나(`factual_error`: true) 구제가 불가능한 수준임.

**[중요] 출력 최적화 규칙:**
1. 평가 결과가 **PASS(수정 불필요)**인 경우:
   - 오직 `null` 이라고만 출력하십시오. (JSON 포맷 아님, 그냥 null 텍스트)
   
2. 평가 결과가 **REVISE** 또는 **DELETE**인 경우:
   - 아래의 상세 JSON 포맷을 작성하여 출력하십시오.

```json
{
  "status": "REVISE" | "DELETE",
  "audit_result": {
    "logic_match": true,
    "missing_exceptions": false,
    "factual_error": false
  },
  "rationale": "평가 이유 (예: 질문은 서명 대상을 묻는데 답변은 의무를 설명함 / '다만'으로 시작하는 단서 조항이 누락됨)",
  "source_citation": "근거 위치 (예: 제2장 2.1.4조 2항)",
  "better_pair": {
    "question": "수정된 질문 (필요시, 원본 유지 가능)",
    "answer": "수정된 답변 (문서의 '단서 조항'까지 포함한 완벽한 답변)"
  }
}
"""